In [2]:
# header
import pandas as pd
import csv
import AddIPAFlex
import json
import MatchHeadwords
from GenerateLexDir import literal_eval_col
from IPython.display import display
from AddIPAFlex import to_ipa

# load in df's
flexicon = pd.read_csv('flexicon-3-20.csv', keep_default_na=False)
senses = pd.read_csv('senses_3_3-FORMATTED.csv', index_col='sense_id', keep_default_na=False)
new_flex = pd.read_csv('Flexport_3_17\\new_flexicon.csv', keep_default_na=False)
new_sns = pd.read_csv('Flexport_3_17\\new_senses.csv', index_col='sense_id', keep_default_na=False)

# take things literally
literal_eval_col(new_flex, 'sense')

In [37]:
# create index using guid
guid = lambda s : s.split(sep='_')[-1][:8]
guids = [ guid(id) for id in flexicon['entry_id'] ]
flexicon.loc[:, 'guids'] = guids
flexicon = flexicon.set_index('guids')
flexicon.head()

guid = lambda s : s.split(sep='_')[-1][:8]
guids = [ guid(id) for id in new_flex['entry_id'] ]
new_flex.loc[:, 'guids'] = guids
new_flex = new_flex.set_index('guids')
new_flex.head()

,headword,entry_id,morph_type,pronunciation,variant_of,these_vars,other_forms,note,sense,date,date_modified
guids,,,,,,,,,,,
0023c0a0,ʃaɾoːp,ʃaɾoːp_0023c0a0-6de7-452a-85b6-0f6e0794014a,stem,,,,,,[e92169c5-ce3e-4a09-9da4-f6860afd0478],2018-06-20T05:09:53Z,2018-06-20T05:09:53Z
002ca005,wao taah,wao taah_002ca005-1366-4a7b-aff1-ac10db10b768,phrase,waʔoʔ ta:h,,,,,[802366cc-fff6-4c35-bff4-a403f4b4130d],2019-01-14T03:46:55Z,2019-01-14T03:54:30Z
0062786c,daheg'õõn,daheg&apos;õõn_0062786c-16c2-4325-b3e4-fac26c1...,stem,dahegõ̰:n,{'6d128d41-81a3-471c-9d02-f855e642415a': {'typ...,,,{'bibliography': 'SIL 2011'},None,2018-03-08T07:17:53Z,2020-02-04T15:32:23Z
0077e8c6,agëëd,agëëd_0077e8c6-ecaf-409e-9076-3ac5e8752c89,stem,,{'agyyt_844f1202-4d2e-4db2-bdb5-08c5213c41af':...,,,,None,2019-11-06T14:32:27Z,2019-11-06T14:32:27Z
007e8774,tame tame,tame tame_007e8774-ab6b-4f2a-b53a-64b144453ab0,phrase,tame tame,,,,{'bibliography': 'SIL 2011'},[a0d825d5-a0ae-402d-8201-6805e39cad0b],2018-03-08T07:17:53Z,2018-06-20T01:46:10Z


In [38]:
removed = new_flex[ [x not in flexicon.index for x in new_flex.index] ]
removed.shape

(854, 11)

In [39]:
new_sns.index

Index(['e92169c5-ce3e-4a09-9da4-f6860afd0478',
       '802366cc-fff6-4c35-bff4-a403f4b4130d',
       'a0d825d5-a0ae-402d-8201-6805e39cad0b',
       '2e7ced09-86a7-4ad6-a7a0-e36f9523d65a',
       '062365d9-1fcd-4828-980d-5fbacad0019b',
       '1040d54b-f002-4f07-826d-01add3578cf8',
       '8ea40d2d-71df-47cf-b1f1-d95a3bdb6476',
       '6e8e07f3-28a9-4adb-b230-f819c944b12a',
       '6c7274f4-8b5b-442d-9e05-9dfbff1f938a',
       '1a7f5cab-b082-42eb-8cac-7781db2f1151',
       ...
       '43f96be7-1ea9-44dd-93fd-c547fd72a028',
       'd0a41f29-94de-4c97-a1ad-81b049e1ede3',
       '4713b0d1-e327-4b22-a41b-7db7430fe70e',
       'b0db2efd-e199-42a1-969e-94d3131b1479',
       '68581f10-629c-47c8-b21e-dc12a51f30a7',
       '7ca6a82d-889c-4ffe-a188-5a4c9ee21eaf',
       '934f9d4e-c6dc-4d59-b337-9b844f0af918',
       '3469ae1b-cc4e-4316-8f5b-f6ad6e92225f',
       'cb2b1a3f-9d04-468c-a497-8395bd836a75',
       '34fe7ac1-e02e-46ae-b874-ca2c7cf8a244'],
      dtype='object', name='sense_id', length=16

In [40]:
for index, row in removed.iterrows():
    sense = row['sense']
    if not sense:
        continue
    for i, id in enumerate(sense[:]):
        sense[i] = [*new_sns.loc[id].values]
removed['sense']

SyntaxError: invalid syntax (<ipython-input-40-3ae87c970f6a>, line 6)

In [41]:
removed.to_csv('removed-entries.csv')

In [5]:
# let's auto-populate some of the entries
removed = pd.read_csv('removed-entries.csv', index_col="entry_id", keep_default_na=False)
removed.head()

,what_to_do,headword,sense,morph_type,pronunciation,note,variant_of,these_vars,other_forms,date,date_modified
entry_id,,,,,,,,,,,
ʃaɾoːp_0023c0a0-6de7-452a-85b6-0f6e0794014a,ignore,ʃaɾoːp,"[['Noun', ""{'en': 'cipó'}"", '', '', '', '']]",stem,,,,,,2018-06-20T05:09:53Z,2018-06-20T05:09:53Z
daheg&apos;õõn_0062786c-16c2-4325-b3e4-fac26c159e80,ignore,daheg'õõn,,stem,dahegõ̰:n,{'bibliography': 'SIL 2011'},{'6d128d41-81a3-471c-9d02-f855e642415a': {'typ...,,,2018-03-08T07:17:53Z,2020-02-04T15:32:23Z
agëëd_0077e8c6-ecaf-409e-9076-3ac5e8752c89,ignore,agëëd,,stem,,,{'agyyt_844f1202-4d2e-4db2-bdb5-08c5213c41af':...,,,2019-11-06T14:32:27Z,2019-11-06T14:32:27Z
ʃɤːw_009b3e07-21aa-4c01-aeda-2447f5bc3e94,ignore,ʃɤːw,"[['Noun', ""{'en': 'pajé (pl sɤ̰ːw)'}"", '', '',...",stem,,,,,,2018-06-20T05:09:53Z,2018-06-20T05:09:53Z
naga̰ːd_00bdbea3-65ff-4c88-bb09-064fdb77ec72,ignore,naga̰ːd,"[['Noun', ""{'en': 'língua'}"", '', '', '', '']]",stem,,,,,,2018-06-20T05:09:54Z,2018-06-20T05:09:54Z


In [ ]:
for index, row in removed.copy().iterrows():
    if row['variant_of'] or 'cartilha' in str(row).lower():
        removed.at[index, 'what_to_do'] = 'ignore'